In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

import os

In [ ]:
os.listdir('open')

In [ ]:
data = pd.read_csv('open/train_features.csv')
label = pd.read_csv('open/train_labels.csv')
display(data)
data.iloc[:, 2:].describe()

In [ ]:
display(label)
plt.figure(figsize=(20, 5))
plt.ylim(0, 125)
label['label'].value_counts().plot.bar()

In [ ]:
unique_ids = data['id'].unique()
rnd_id = np.random.choice(unique_ids)
rnd_data = data[data['id']==rnd_id]

fig, axes = plt.subplots(6, 1, figsize=(10, 5))
print(label[label['id']==rnd_id].iloc[0, 2])
for i, ax in enumerate(axes):
    ax.plot(rnd_data.iloc[:, 2+i])

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':.6f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        self._list = []

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        self._list.append(self.avg)

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)
    
def get_inout_seq(seq, len_seq, len_pred):
    inout_seq = []
    L = len(seq)
    for i in range(0, L-len_pred, len_pred):
        input_seq = seq

In [ ]:
class GRU_classifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, batch_size, device):
        super().__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.device = device
        
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=False, dropout=0.)
        self.relu = nn.ReLU()
        self.dense1 = nn.Linear(hidden_size, output_size*2)
        self.dense2 = nn.Linear(output_size*2, output_size)
        self.softmax = nn.Softmax(dim=0)
        
    def forward(self, input_):
        hidden = self.init_hidden()
        input_ = input_.view(len(input_), self.batch_size, -1)
        output, hidden = self.gru(input_, hidden)
        output = self.relu(output[-1, :, :].squeeze())
        output = self.dense1(output)
        output = self.relu(output)
        output = self.dense2(output)
        output = self.softmax(output)
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(self.num_layers, self.batch_size, self.hidden_size, device=self.device)
    

class Exercise_dataset(Dataset):
    def __init__(self, df, label, device):
        self.df = df
        self.label = label
        self.device = device
        self.unique_ids = self.df['id'].unique()
        self.unique_labels = self.label['label'].unique()
        
    def __len__(self):
        return len(self.unique_ids)
    
    def __getitem__(self, idx):
        idx = idx % len(self.unique_ids)
        id_ = self.unique_ids[idx]
        
        sample = dict()
        sample['seq'] = torch.Tensor(self.df[self.df['id']==id_].iloc[:, 2:].values, device=self.device)
#         sample['label'] = torch.Tensor(self.enc.transform([[idx]]), device=self.device)
        sample['label'] = torch.Tensor([self.label[self.label['id']==id_].iloc[0, 1]], device=self.device)
        
        return sample

In [ ]:
BATCH_SIZE = 1
N_EPOCHS = 30
VIS_FREQ = 1
N_CLASSES = len(label['label'].unique())

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

unique_ids = data['id'].unique()
cv = KFold(n_splits=5,
           random_state=42,
           shuffle=True)

for k, (train_id, valid_id) in enumerate(cv.split(unique_ids)):
    print(f'Fold {k} ' + '='*50)
    X_train = data[data['id'].isin(train_id)]
    y_train = label[label['id'].isin(train_id)]
    X_valid = data[data['id'].isin(valid_id)]
    y_valid = label[label['id'].isin(valid_id)]
    
    train_dataset = Exercise_dataset(X_train, y_train, device)
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
    valid_dataset = Exercise_dataset(X_valid, y_valid, device)
    valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

    model = GRU_classifier(input_size=6,
                           hidden_size=4,
                           num_layers=1,
                           output_size=len(label['label'].unique()),
                           batch_size=1,
                           device=device)

    loss_CE = nn.CrossEntropyLoss()
    optim = torch.optim.AdamW(model.parameters(), lr=1e-03, weight_decay=1e-02)
    
    loss_train = AverageMeter('loss_train', ':.6f')
    loss_valid = AverageMeter('loss_valid', ':.6f')

    for epoch in range(N_EPOCHS):
        for i, (train_sample, valid_sample) in enumerate(zip(train_dataloader, valid_dataloader)):
            train_seq = train_sample['seq']
            train_label = train_sample['label']
            train_seq = train_seq.transpose(0, 1)
            train_label = train_label.transpose(0, 1).squeeze(1).long()
            
            optim.zero_grad()
            
            pred, hidden = model(train_seq)
            
            if pred.size(0)==N_CLASSES:
                pred = pred.unsqueeze(0)
#             if train_label.size(0)==1:
#                 train_label = train_label.unsqueeze(0)
                
#             print(f'pred : {pred.size()}')
#             print(f'label : {train_label.size()}')
#             print(pred)
#             print(train_label)
            loss = loss_CE(pred, train_label)
            loss_train.update(loss.item())
            
            loss.backward()
            optim.step()
            
            with torch.no_grad():
                valid_seq = valid_sample['seq']
                valid_label = valid_sample['label']
                valid_seq = valid_seq.transpose(0, 1)
                valid_label = valid_label.transpose(0, 1).squeeze(1).long()
                
                pred, hidden = model(valid_seq)
                
                if pred.size(0)==N_CLASSES:
                    pred = pred.unsqueeze(0)
                    
                loss = loss_CE(pred, valid_label)
                loss_valid.update(loss.item())
        
        if epoch % VIS_FREQ == 0:
            print(f'Epoch[{epoch}] : {loss_train}, {loss_valid}')

In [ ]:
def show_result():
    plt.figure(figsize=(20, 10))
    plt.plot(loss_train.avg_list, color='b', label='loss_train')
    plt.plot(loss_valid.avg_list, color='r', label='loss_valid')
    plt.legend()
    plt.show()
        
show_result()